In [2]:
import xml.etree.ElementTree
import pandas as pd
import time
import datetime
now = datetime.datetime.now()
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import re
from collections import Counter
%matplotlib inline  

import os
import random
import collections
import sys
import gensim
import stanza
import matplotlib.pyplot as plt 
import wordcloud

plt.rcParams['figure.figsize'] = (10, 15)
from feel_it import EmotionClassifier, SentimentClassifier

from datetime import date, timedelta


In [3]:
emotion_classifier=EmotionClassifier()
sentiment_classifier=SentimentClassifier()

In [4]:


def cleanTxt(text):
    text=re.sub(r'@[A-Za-z0-9]+','',text) # remoeved @mentions
    text=re.sub(r'@[A-Za-z0-9]+','',text) # remoeved @mentions
    text=re.sub(r"\'","'",text)
    text=re.sub(r'#', '', text)
    text=re.sub(r'RT[\s]+','',text)
    text=re.sub(r'https?:\/\/\S+','',text)
    text=re.sub("\r\n",' ',text)
    text=re.sub("\n","",text)
    text=re.sub("\xa0","",text)
    text=re.sub("\n\n",' ',text)
    text=re.sub("\t",' ',text)
    text=re.sub("\r",' ',text)
    text=re.sub("<*",'',text)
    #text=re.sub("*<",'',text)
    text=re.sub(">*",'',text)
    #text=re.sub("*>",'',text)
    text=re.sub(".Twitter","",text)
    text=re.sub(".twitter","",text)
    
    return text

# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 

def clean_n(text):
    text=re.sub(r"\n","",text)

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

#def remove_tags(text):
#    return ''.join(xml.etree.ElementTree.fromstring(text).itertext())

def remove_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def avg_sentence_len(text):
  sentences = text.split(".") #split the text into a list of sentences.
  words = text.split(" ") #split the input text into a list of separate words
  if(sentences[len(sentences)-1]==""): #if the last value in sentences is an empty string
    average_sentence_length = len(words) / len(sentences)-1
  else:
    average_sentence_length = len(words) / len(sentences)
  return average_sentence_length #returning avg length of sentence
  
def daterange(start_date, end_date):
        for n in range(int((end_date - start_date).days)):
            yield start_date + timedelta(n)    

def extracting_urls_from_repubblica(start_date):
    
    url_distributed_in_time=[]
    count=0

    stopword_df=pd.read_csv('parole_ita2.csv',sep=';')
    words=stopword_df['cosa']

    for j in range (0,455):
        count=count+1
        if count<=11:
            if (int(start_date[5:7])+1)<=9:
                start_date=str(start_date[:4])+'-0'+str(int(start_date[5:7])+1)+'-'+str(start_date[8:])
                #print(start_date)
            else: 
                start_date=str(start_date[:4])+'-'+str(int(start_date[5:7])+1)+'-'+str(start_date[8:])
                #print(start_date)
        else: 
            count=0
            start_date=str(int(start_date[:4])+1)+'-01-01'
            #print(start_date)

        random_words=[]

        for l in range (0,20):
            random_words.append(random.choice(words))
        print (random_words)


        for i in random_words:

            source_url='https://ricerca.repubblica.it/ricerca/repubblica?query='+i+'&fromdate='+start_date+'&todate=2022-12-18&sortby=adate&author=&mode=any'
            print(start_date)
            x = requests.get(source_url)
            soup = BeautifulSoup(x.content, "html.parser")
            headers=soup.find_all('h1')
            time.sleep(5)
            #print(soup)

            for i in headers:
                #print(i)
                url=str(i)
                url=url.split()[1]
                #url=url[:-1]

                if url[:5]=='https':
                    url_distributed_in_time.append(url)
                    print(url)
                elif url[:6]=='href="':
                    url='https:'+url[6:]
                    url=url[:-1]
                    print (url)
                    url_distributed_in_time.append(url)

                else: 
                    pass

        conto=0
        mylist=url_distributed_in_time
        mylist2=[]
        mylist3=[]
        mylist4=[]
        mylist5=[]

    #cleaning urls

        for i in mylist:
            if i.startswith('https:https'): 
                conto=conto+1
                mylist2.append(i.replace('https:https','https:'))

            elif i.startswith('https:http'):
                mylist2.append(i.replace('https:http','https:'))

            else:
                mylist2.append(i)

        for i in mylist2:

            if i.startswith('https::'):
                mylist3.append(i.replace('https::','https:'))

            else:
                mylist3.append(i)

        for i in mylist3:
            if i.startswith('http'):
                mylist4.append(i[:-1])

        for i in mylist4:
            if i!='https://quotidiano.repubblica.it/edicola/catalogogenerale.jsp?ref=search':
                mylist5.append(i)

    #writing urls on file

        f = open("url_la_repubblica.txt", "a")

        for i in mylist5:
            f.write(i)
            f.write('\n')
        f.close()
        
        repubblica_urls=mylist5

def extracting_urls_from_fattoquotidiano(start_date):
    
    count=0
    selected_dates=[]
    start_date = date(2022, 12, 30)
    end_date = date(2022, 12, 31)
    for single_date in daterange(start_date, end_date):
        selected_dates.append(single_date.strftime("%Y/%m/%d"))
        
    for j in selected_dates:
        source_url='https://www.ilfattoquotidiano.it/'+j
        print(j) 
        x = requests.get(source_url)
        soup = BeautifulSoup(x.content, "html.parser")
        headers=soup.find_all('h3', {"class": "p-item"})
        #time.sleep(5) 
        #print(soup)

        count=count+1
        url_distributed_in_time_fq=[]

        for i in headers:
            url=str(i)
            #print(url)
            try: 
                url = url.split()[2]
            except IndexError:
                url = 'null'
            url_distributed_in_time_fq.append(url[6:-1])
            print(url[6:-1])

    fattolist=[]
    fattolist = list(dict.fromkeys(url_distributed_in_time_fq))
    len(fattolist)
    fatto_urls=[]
    with open('/Users/gabrieledettori/PROGETTOTESI-1.0.0/url_fatto_quotidiano.txt', 'w') as f:
        for i in mylist:
            print(i)
            if i.startswith('http'):
                f.write(i[:-1])
                fatto_urls.append(i[:-1])
                f.write('\n')

    f.close()
    
def extracting_articles_from_repubblica(repubblica_urls):
    index=0

    for i in range (0,len(repubblica_urls)):
        index=i
        title=''

        # Fill in your details here to be posted to the login form.
        payload = {
            'inUserName': '4gwpk6gsqj@privaterelay.appleid.com',
            'inUserPass': 'WebScarpingProject'
        }

        # Use 'with' to ensure the session context is closed after use.
        with requests.Session() as s:
            p = s.post('https://login.gedi.it/clp/login.php?gig_ssoToken=eu1_tk1.euY-11eQdFkqS2e7vj_s-Pt0Xl1p2oVbdfxZgWHjG9Q&gig_brand=repubblica', data=payload)
            article_url=(mylist[index][:-11])

            if article_url in already_ext or article_url.startswith('https::'): 

                print ('Already extracted: ',article_url, ' - ', index)
                pass

            else:

                print ('Proceding with extraction')

                try:
                    x = s.get(mylist[index][:-11])
                except NameError:
                    pass

                soup = BeautifulSoup(x.content, "html.parser")
                contatore=0
                title=soup.find_all('title')
                title=(str(title).replace('<title>','').replace('</title>',''))
                paragraph=soup.find_all('p')
                paragraph=(str(paragraph).replace('<p>','').replace('</p>',''))
                match = re.search(r'\d{4}/\d{2}/\d{2}', mylist[index][:-11])

                if match is None:
                    data=datetime
                else:
                    data = datetime.datetime.strptime(match.group(), '%Y/%m/%d').date()
                author=cleanhtml(str(soup.find_all('em')))[:-1]

                title=str(title).replace('/','')

                with open('/Users/gabrieledettori/PROGETTOTESI-1.0.0/simetadata/'+str(title)[:150]+'.txt', 'w') as f:

                    f.write(paragraph)

                    art_title.append(title)
                    art_data.append(data) 
                    art_author.append(author)
                    art_url.append(mylist[index][:-11])
                    index_list.append(index)
                    paragraphs.append(paragraph)

                    f.close()

                print('Titolo:',title)
                print(paragraph[:500])
                print('data: ',data,' author: ', author,' url: ', mylist[index][:-11])
                print(datetime.datetime.now())
                print('INDEX: ',index)

                print('Process: ', (index/len(mylist)*100),'%')

        time.sleep(0.45)

        if '[ERROR: The request could not be satisfied]' in title:
            print('Started sleeping for 100 at :', datetime.datetime.now())
            time.sleep(100)
            print('Stopped sleeping at :', datetime.datetime.now())

            pass

    repubblica_frame=({'index':index_list,'title':art_title,'data':art_data,'url':art_url,'author':art_author,'paragraph':paragraphs})
    repubblica_df=pd.DataFrame(repubblica_frame)
    repubblica_df

    
def extracting_articles_from_fatto(fatto_urls):
    
    len_mylist=len(fatto_urls)

    for i in range (0,len_mylist):

        payload = {
            'inUserName': 'gabriele.dettori@vodafone.com',
            'inUserPass': 'Caffelatte1*'
        }

            # Use 'with' to ensure the session context is closed after use.
        with requests.Session() as s:
            p = s.post('https://shop.ilfattoquotidiano.it/login/', data=payload)

            try:
                x = s.get(mylist[i])
            except NameError:
                pass

            soup = BeautifulSoup(x.content, "html.parser")
            contatore=0
            title=soup.find('h1', {"class": "title-article"})
            paragraph=soup.find_all('section', {"class": "article-content"})
            title=(str(title).replace('<h1 class="title-article">','').replace('</h1>','')).replace ('<br/>','').replace('/','')
            paragraph=(str(paragraph).replace('[<section class="article-content">','').replace('</section>]',''))
            paragraph=(str(paragraph).replace('<p>','').replace('</p>',''))
            paragraph=cleanhtml(paragraph)

            #print(title)
            #print(paragraph)

            #title=(str(title).replace('<title>','').replace('</title>',''))
            #paragraph=soup.find_all('p')
            #paragraph=(str(paragraph).replace('<p>','').replace('</p>',''))
            match = re.search(r'\d{4}/\d{2}/\d{2}', mylist[i])
            #print(soup)
            if match is None:
                data=datetime
            else:
                data = datetime.datetime.strptime(match.group(), '%Y/%m/%d').date()
                author=cleanhtml(str(soup.find_all('em')))[:-1]
            #print(data)

            with open('/Users/gabrieledettori/PROGETTOTESI-1.0.0/fattoquotidiano/'+str(title)[:150]+'.txt', 'w') as f:


                f.write(paragraph)

                art_title.append(title)
                art_data.append(data) 
                art_author.append(author)
                art_url.append(mylist[i])
                paragraphs.append(paragraph)

                f.close()
            print('Titolo:',title)
            print(paragraph[:500])
            print('data: ',data,' author: ', author,' url: ', mylist[i])
            print(datetime.datetime.now())
            print('INDEX: ',i)

            print('Process: ', round(i/len_mylist*100,2),'%')

    finalframe=({'title':art_title,'data':art_data,'url':art_url,'author':art_author,'paragraph':paragraphs})
    frame_final=pd.DataFrame(finalframe)
    frame_final['data'] = pd.to_datetime(frame_final['data'])
    frame_final_sorted=frame_final.sort_values(by=['data'])
    
    art_title=[]
    art_data=[]
    art_url=[]
    art_author=[]
    paragraphs=[]
    
    for i in range (0,len(fatto1['paragraph'])):
        art_title.append(fatto1['title'][i])
        art_data.append(fatto1['data'][i])
        art_url.append(fatto1['url'][i])
        art_author.append(fatto1['author'][i])
        paragraphs.append(cleanTxt(cleanhtml(fatto1['paragraph'][i])))

    for i in range (0,len(fatto2['paragraph'])):
        art_title.append(fatto2['title'][i])
        art_data.append(fatto2['data'][i])
        art_url.append(fatto2['url'][i])
        art_author.append(fatto2['author'][i])
        paragraphs.append(cleanTxt(cleanhtml(fatto2['paragraph'][i])))
        
    #cleaner

    cleaned_paragraphs=[]
    cleaned_titles=[]
    for i in range (0,len(paragraphs)):
        if 'jwplayer' in str(paragraphs[i]):
            toremove=fattoquotidiano_sorted['paragraph'][i].find('jwplayer')
            toremove2=fattoquotidiano_sorted['paragraph'][i].find('     });')
            #toremove3=fattoquotidiano_sorted['paragraph'][i][toremove:toremove2]
            #problemms.append(i)
            #dateprob.append(fattoquotidiano_sorted['data'][i])
            cleaned_paragraph=paragraphs[i][:toremove] + paragraphs[i][toremove2:]
            cleaned_paragraphs.append(cleaned_paragraph)

        else:
            cleaned_paragraphs.append(fattoquotidiano_sorted['paragraph'][i])

    for i in range (0,len(art_title)):
        if art_title[i] is not None:
            cleaned_title=(cleanTxt(cleanhtml(str(art_title[i]))))
            cleaned_titles.append(cleaned_title)
        else:
            cleaned_titles.append('')

    paragraphs=cleaned_paragraphs
    art_title=cleaned_titles
        
    fattoframe=({'title':art_title,'data':art_data,'url':art_url,'author':art_author,'paragraph':paragraphs})
    fatto_df=pd.DataFrame(fattoframe)
    fatto_df


In [11]:
#starding date of la repubblica
start_date='1985-01-01'

In [26]:
#starding date of il fatto quotidiano
start_date='2009/08/08/'

In [2]:
fattodf=pd.read_csv("/Users/gabrieledettori/PROGETTOTESI-1.0.0/fattoframecleaned3-enriched.csv")
fattodf

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,title,data,url,author,paragraph,nummber_dots,number_commas,number_e,frasi_splitted_dot,num_frasi_sep_by_dot,frasi_splitted_comma,num_frasi_sep_by_comma,frasi_splitted_duepunti,num_frasi_sep_by_duepunti,frasi_semplici,frasi_nominali
0,0,0,0,"Medio Oriente, il premier israeliano “La guer...",2010-09-01,https://www.ilfattoquotidiano.it/2010/09/01/me...,[,Segnali distensivi per il conflitto tra israel...,0,1,0,NaN,0,"['Medio Oriente', ' il premier israeliano “La...",2,NaN,0,Sì,Sì
1,1,1,1,Negli Stati Uniti il lobbismo dell’industria p...,2010-09-01,https://www.ilfattoquotidiano.it/2010/09/01/ne...,[,Nell’anno del peggior disastro ecologico della...,0,0,0,NaN,0,NaN,0,NaN,0,Sì,Sì
2,2,2,2,"Botta e risposta fra Santoro-Masi, Annozero an...",2010-09-01,https://www.ilfattoquotidiano.it/2010/09/01/sa...,[,Michele Santoro è stanco dei continui ritardi ...,0,1,1,NaN,0,"['Botta e risposta fra Santoro-Masi', ' Annoze...",2,NaN,0,Sì,Sì
3,3,3,3,Abuso di auto blu: il presidente del consiglio...,2010-09-01,https://www.ilfattoquotidiano.it/2010/09/01/ab...,[,Abuso di auto blu in Friuli Venezia Giulia nel...,0,0,0,NaN,0,NaN,0,"['Abuso di auto blu', ' il presidente del cons...",2,Sì,Sì
4,4,4,4,"Lungaggini e fondi fantasmi, l’Expo incubo pe...",2010-09-01,https://www.ilfattoquotidiano.it/2010/09/01/lu...,[Il Tempo,"Expo 2015 potrebbe trasferirsi a Smirne, sulla...",1,1,1,"['Lungaggini e fondi fantasmi, l’Expo incubo ...",2,"['Lungaggini e fondi fantasmi', ' l’Expo incub...",2,NaN,0,Sì,Sì
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44968,44968,44970,44978,"Manovra, come cambia la spesa dello Stato: più...",2022-12-30,https://www.ilfattoquotidiano.it/2022/12/30/m5...,"[Adnkronos, Adnkronos",Per i grandi capitoli che incidono più diretta...,0,3,2,NaN,0,"['Manovra', ' come cambia la spesa dello Stato...",4,"['Manovra, come cambia la spesa dello Stato', ...",2,Sì,Sì
44969,44969,44971,44979,"Siccità, alluvioni e temperature record: il 20...",2022-12-30,https://www.ilfattoquotidiano.it/2022/12/30/pe...,[,Il 2022 ripercorso attraverso le immagini dei ...,1,1,1,"['Siccità, alluvioni e temperature record: il ...",2,"['Siccità', ' alluvioni e temperature record: ...",2,"['Siccità, alluvioni e temperature record', ' ...",2,No,No
44970,44970,44972,44980,"Trump, conti correnti in Cina e zero beneficen...",2022-12-30,https://www.ilfattoquotidiano.it/2022/12/30/pu...,"[LA GENESI, LE REAZIONI IN AULA: VOTI PRO E CO...",L’ultima volta in cui era avvenuta una cosa si...,0,1,1,NaN,0,"['Trump', ' conti correnti in Cina e zero bene...",2,"['Trump, conti correnti in Cina e zero benefic...",2,Sì,Sì
44971,44971,44973,44981,Il governo vieta il rilascio di certificati an...,2022-12-30,https://www.ilfattoquotidiano.it/2022/12/30/br...,[,Stop al rilascio di certificati anagrafici per...,1,0,1,['Il governo vieta il rilascio di certificati ...,2,NaN,0,NaN,0,Sì,Sì


In [3]:
repubblicadf=pd.read_csv("/Users/gabrieledettori/PROGETTOTESI-1.0.0/clean_frame_final3-enriched.csv")
repubblicadf

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,title,data,url,author,paragraph,ratio,corpus,...,number_commas,number_e,frasi_splitted_dot,num_frasi_sep_by_dot,frasi_splitted_comma,num_frasi_sep_by_comma,frasi_splitted_duepunti,num_frasi_sep_by_duepunti,frasi_semplici,frasi_nominali
0,0,0,0,LASSU' SULLA MONTAGNA IL MONDO CAMBIA FACCIA?,1985-02-01,https://ricerca.repubblica.it/repubblica/archi...,nostro inviato,[BORMIO (l.c.) - L Argentina ha tremila chilom...,0.633,608,...,0,0,NaN,0,NaN,0,NaN,0,Sì,Sì
1,1,1,1,"LA RADIO, IL SUO PRESENTE, IL SUO FUTURO",1985-02-01,https://ricerca.repubblica.it/repubblica/archi...,NaN,[ROMA - Parliamo tanto di radio. La radio pubb...,0.591,536,...,2,0,NaN,0,"['LA RADIO', ' IL SUO PRESENTE', ' IL SUO FUTU...",3,NaN,0,Sì,Sì
2,2,2,2,'DE MICHELIS FACCIA L' AUTOCRITICA',1985-02-01,https://ricerca.repubblica.it/repubblica/archi...,GIUSTINO FABRIZIO,[ROMA - Il Psi tenta di ricomporre il dissidio...,0.578,721,...,0,0,NaN,0,NaN,0,NaN,0,Sì,Sì
3,3,3,3,L' AMBASCIATORE LIBICO ACCUSA 'NON AVETE DIFES...,1985-02-01,https://ricerca.repubblica.it/repubblica/archi...,MAGDI ALLAM,"[ROMA - Abdurrahman Mohamed Shalgam, segretari...",0.569,575,...,0,0,NaN,0,NaN,0,NaN,0,Sì,Sì
4,4,4,4,"NIENTE PAURA, AMICI A QUEL DIAVOLO CI PENSIAMO...",1985-02-01,https://ricerca.repubblica.it/repubblica/archi...,TULLIO KEZICH,[CHI HA PAURA dei fantasmi cattivi? L America ...,0.599,812,...,1,0,NaN,0,"['NIENTE PAURA', ' AMICI A QUEL DIAVOLO CI PEN...",2,NaN,0,Sì,Sì
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44038,44038,44038,44038,"""Scarpette rotte"": la firma di Emma Dante per ...",2022-12-11,https://ricerca.repubblica.it/repubblica/archi...,"[Cerca,",[ di Giulio Baffi Primo dei tre spettacoli del...,0.338,452,...,0,0,NaN,0,NaN,0,"['""Scarpette rotte""', ' la firma di Emma Dante...",2,Sì,Sì
44039,44039,44039,44039,"I leoni di Regragui ""Siamo in missione per con...",2022-12-11,https://ricerca.repubblica.it/repubblica/archi...,"[Cerca, Dal Nostro Inviato Matteo Pinci","[ DOHA — Quando escono dallo spogliatoio, can...",0.327,1201,...,0,0,NaN,0,NaN,0,NaN,0,Sì,Sì
44040,44040,44040,44040,Il manifesto dei diritti dei bambini va in sce...,2022-12-11,https://ricerca.repubblica.it/repubblica/archi...,NaN,[ di Gilda Camero Un viaggio fantastico dedica...,0.343,562,...,0,0,NaN,0,NaN,0,NaN,0,Sì,Sì
44041,44041,44041,44041,Lotta alla plastica e niente fumo Scommessa ‘...,2022-12-11,https://ricerca.repubblica.it/repubblica/archi...,NaN,"[ di Bettina Bush Riciclo rifiuti, lotta alla ...",0.295,1234,...,0,1,NaN,0,NaN,0,NaN,0,Sì,Sì
